In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

In [2]:
m_codes = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighborhood'], index=[0])

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
with urllib.request.urlopen(url) as response:
    wikipage = response.read()
    markup = BeautifulSoup(wikipage, 'html.parser')

table = markup.find_all('tbody')[0]

row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        m_codes.iat[row_marker-1, column_marker] = column.get_text() #subtract 1 from row_marker or else row zero is NaNs
        column_marker += 1
    m_codes.loc[row_marker+1] = ['' for n in range(3)]
    row_marker += 1
    
m_codes['Neighborhood'] = m_codes['Neighborhood'].str.strip('\n')

m_codes.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
m_codes = m_codes[m_codes['Borough'] != "Not assigned"]

m_codes.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
m_codes['Neighborhood'] =  m_codes.groupby(by=['Postal Code', 'Borough']).transform(lambda x: ', '.join(x))
m_codes.drop_duplicates(inplace=True)

m_codes.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Not assigned


In [6]:
#m_codes['Neighborhood'] = np.where(m_codes['Neighborhood'] == "Not assigned", m_codes['Borough'], m_codes['Neighborhood'] )

m_codes.loc[m_codes['Neighborhood'] == "Not assigned", 'Neighborhood'] = m_codes['Borough']

m_codes.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


In [7]:
m_codes.shape

(104, 3)